In [10]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models

import nltk

In [20]:
class Embeddings():
    def __init__(self, vocab, emd_dims):
        self.vocab = vocab
        self.embeds = nn.Embedding(len(vocab), emd_dims)
    
    def get_embedding(self, word):
        lookup_tensor = torch.tensor(self.vocab[word], dtype = torch.long)
        return self.embeds(lookup_tensor)
    
    def vocab_size():
        return len(self.vocab.keys())

In [3]:
test_df = pd.read_csv('./data/coco/coco_test_all.csv')

In [4]:
data_folder = ['coco','bing','flickr']
dataframes = []
for f in data_folder:
    files = os.listdir('./data/'+f)   
    for path in files:
        if('.csv' in path):
            csv_path = './data/'+f+'/'+path
            print(csv_path)
            df = pd.read_csv(csv_path)
            dataframes.append(df)
len(dataframes)

./data/coco/coco_test_all.csv
./data/coco/coco_val_all.csv
./data/coco/coco_train_all.csv
./data/bing/bing_train_all.csv
./data/bing/bing_val_all.csv
./data/bing/bing_test_all.csv
./data/flickr/flickr_train_all.csv
./data/flickr/flickr_val_all.csv
./data/flickr/flickr_test_all.csv


9

In [5]:
df = pd.concat(dataframes, axis = 0)
len(df)

14815

In [8]:
questions = list(df['questions'])
freq = {}
for q in questions:
    for question in q.split('---'):
        wordlist = nltk.word_tokenize(question)
        for word in wordlist:
            if(word not in freq):
                freq[word] = 1
            else:
                freq[word] += 1
len(freq)

12098

In [9]:
vocab = {}
counter = 0
for key in freq.keys():
    if freq[key]>=3:
        vocab[key] = counter
        counter += 1


vocab['<unk>'] = counter
len(vocab)

5061

In [ ]:
# features = list(model.features)[-1]
# m = nn.ModuleList(features).eval()
model.features

In [21]:
class Vqgnet(nn.Module):
    def __init__(self, num_lstm_layers, embedding, max_len):
        super(Vqgnet, self).__init__()
        self.embedding = embedding
        self.model_vgg = models.vgg19(pretrained=True)
        for p in self.model_vgg.parameters():
            p.requires_grad = False
        self.features = self.model_vgg.classifier[:-1]
        self.transform_layer = nn.Linear(4096, 512)
        self.feature_to_word = nn.Linear(512, self.embedding.vocab_size())
        self.n_lstm_layers = n_lstm_layers
        self.lstm = nn.LSTM(512, 512)
        self.max_len = max_len
    
    def forward(image, question):
        # teacher forcing using gt question
        
        # getting image features
        x = self.features(image)
        x = F.relu(x)
        x = F.relu(self.transform_layer(x))
        
        cell_state = torch.randn(1, 1, 512)
        que = []
        # embedding phase
        for i in range(len(question)):
            word = question[i]
            embed = self.embedding.get_embedding(word)
            output, cell_state = self.lstm(embed, cell_state)
            que.append(output)
    
    def test(image):
        # get image features
        x = self.features(image)
        x = F.relu(x)
        x = F.relu(self.tranform_layer(x))
        
        cell_state = torch.randn(1, 1, 512)
        que = []
        output = ''
        predicted_question = []
        # generate question
        while output != self.embedding.get_eoq():
            x, cell_state = self.lstm(x, cell_state)
            output = F.softmax(x)
            predicted_question.append(output)
        
        return predicted_question
        

In [17]:
model = models.vgg19(pretrained=True).eval()

In [18]:
new_model = model.classifier[:-1]

In [19]:
new_model

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
)